In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from xgboost import XGBRegressor

Import DataSet

In [6]:
fitness=pd.read_csv("C:\\UOC pdf\\3rd Year\\MachineLearning-01\\final_project\\healthFitnessDataset.csv")

In [8]:
X=fitness.drop(columns=['fitness_level'])
Y=fitness['fitness_level']

Split Data Training and Testing

In [11]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.7,random_state=42)
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
Y_train = Y_train.reset_index(drop = True)
Y_test = Y_test.reset_index(drop = True)

Scale Numerical Variables

In [14]:
scaler=StandardScaler()

num_cols=X_train.select_dtypes(include='number').columns
X_train[num_cols]=scaler.fit_transform(X_train[num_cols])

In [16]:
##categorical Data Encoding using Ordinal and One-Hot Encoding

##Ordinal Endcoding
# Define the order manually
intensity_order = ['Low','Medium','High']
# Create encoder
ordinal_encoder = OrdinalEncoder(categories=[intensity_order])
# Fit and transform
X_train['intensity']= ordinal_encoder.fit_transform(X_train[['intensity']])

In [18]:
##Nominal Endcoding
# Create encoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Fit and transform
encoded_data = onehot_encoder.fit_transform(X_train[['gender','activity_type','smoking_status']])

# Convert the encoded array to a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=onehot_encoder.get_feature_names_out(['gender', 'activity_type', 'smoking_status']))

# Combine with the original dataset
X_train = X_train.drop(['gender','activity_type','smoking_status'], axis=1)
X_train = pd.concat([X_train.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

In [20]:
##Scaling,Endcoding Testing set

##Scaling
X_test[num_cols]=scaler.fit_transform(X_test[num_cols])

In [22]:
##Endcoding categorical variables

###Nominal Endcoding
# Create encoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Fit and transform
encoded_data = onehot_encoder.fit_transform(X_test[['gender','activity_type','smoking_status']])

# Convert the encoded array to a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=onehot_encoder.get_feature_names_out(['gender', 'activity_type', 'smoking_status']))

# Combine with the original dataset
X_test = X_test.drop(['gender','activity_type','smoking_status'], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

In [24]:
###Ordinal Endcoding
# Define the order manually
intensity_order = ['Low','Medium','High']
# Create encoder
ordinal_encoder = OrdinalEncoder(categories=[intensity_order])
# Fit and transform
X_test['intensity']= ordinal_encoder.fit_transform(X_test[['intensity']])

Fit XGBoost Model

In [39]:
xgb=XGBRegressor()

In [41]:
xgb.fit(X_train,Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [43]:
Y_pred_xgb=xgb.predict(X_test)
Y_pred_train_xgb=xgb.predict(X_train)

In [52]:
mae_test_xgb=mean_absolute_error(Y_test,Y_pred_xgb)
mae_train_xgb=mean_absolute_error(Y_train,Y_pred_train_xgb)

print("Testing MAE:",mae_test_xgb)
print("Training MAE:",mae_train_xgb)

Testing MAE: 1.7771951979696203
Training MAE: 1.1350822121486526


In [56]:
mse_test_xgb=mean_squared_error(Y_test,Y_pred_xgb)
mse_train_xgb=mean_squared_error(Y_train,Y_pred_train_xgb)

print("Testing MSE:",mse_test_xgb)
print("Training MSE:",mse_train_xgb)

Testing MSE: 5.266108450723383
Training MSE: 2.2498867297448326


In [58]:
print("R2 score:",r2_score(Y_pred_xgb,Y_test))

R2 score: 0.7954864228958313


Fit XGBoost Model with Hyperparameter Tuninng(GridSerach Method)

In [89]:
param_grid_xgb = {
    'n_estimators': [100, 200,300],
    'max_depth': [3, 5, 7,10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9, 1],
}
xgb_cv=GridSearchCV(estimator=xgb,param_grid=param_grid_xgb,cv=5,n_jobs=-1)

In [91]:
xgb_cv.fit(X_train,Y_train)

C:\Users\ASUS\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7, 10],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.7, 0.8, 0.9, 1]})

In [101]:
xgb_cv.best_params_

{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 300, 'subsample': 1}

In [93]:
Y_pred_xgb_cv=xgb_cv.predict(X_test)
Y_pred_train_xgb_cv=xgb_cv.predict(X_train)

In [95]:
test_mae_xgb_cv=mean_absolute_error(Y_pred_xgb_cv,Y_test)
train_mae_xgb_cv=mean_absolute_error(Y_pred_train_xgb_cv,Y_train)

print("Testing MAE(HyperParameter Tuninng):",test_mae_xgb_cv)
print("Training MAE(Hyperparameter Tuninng):",train_mae_xgb_cv)

Testing MSE(HyperParameter Tuninng): 1.7159714144571425
Training MSE(Hyperparameter Tuninng): 0.44138357735706657


In [97]:
mse_test_xgb_cv=mean_squared_error(Y_pred_xgb_cv,Y_test)
mse_train_xgb_cv=mean_squared_error(Y_pred_train_xgb_cv,Y_train)

print("Mean Square Error(Hyper Parameter Tuninng):",mse_test_xgb_cv)
print("Mean Sqaure Error(Hyper Parameter Tuninng):",mse_train_xgb_cv)

Mean Square Error(Hyper Parameter Tuninng): 4.939491979730067
Mean Sqaure Error(Hyper Parameter Tuninng): 0.3824259362198115


In [99]:
r_2Score_cv=r2_score(Y_test,Y_pred_xgb_cv)
print("R2 score(HyperParameter Tuninng):",r_2Score_cv)

R2 score(HyperParameter Tuninng): 0.8392296111705586
